In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

NUM_PROGRAM = 8

cut_programs = np.load('cut_Programs.npy')
cut_questions = np.load('cut_Questions.npy')

voc_dict = np.load('voc_dict.npy')

#print(cut_programs[0][0])
sentences = []
tag = 0
for program in cut_programs:
    for episode in program:
        for sentence in episode:
            sentences.append(TaggedDocument(sentence, [tag]))
            tag = tag+1
            
#print(sentences[:5])    
print(len(sentences))         

D:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


2396906


In [2]:
print(cut_questions[2][0])

# 6 optional reponses
#for i in range(1, 7):
#    print(cut_questions[0][i])

[['你', '說', '我們', '做', '父母', '的', '最', '擔心', '的', '就是', '這個'], []]


In [4]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from gensim.test.utils import get_tmpfile
import multiprocessing

cores = multiprocessing.cpu_count()

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, vector_size=100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores, alpha=0.05, comment='alpha=0.05'),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    #Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=0, epochs=20, workers=cores),
]

for model in simple_models:
    model.build_vocab(sentences)
    print("%s vocabulary scanned & state initialized" % model)
    
for i in [0,1]:
    model = simple_models[i]
    print("Training %s" % model)
    %time model.train(doc_list, total_examples=len(doc_list), epochs=model.epochs)
    model.save("my_doc2vec_simplemodel_%(i)")

Doc2Vec(dbow,d100,n5,mc2,t4) vocabulary scanned & state initialized
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4) vocabulary scanned & state initialized
Training Doc2Vec(dbow,d100,n5,mc2,t4)


NameError: name 'doc_list' is not defined

Training Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4)


NameError: name 'doc_list' is not defined

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

doc_words1 = ['對', ' ', '就是', '這個', ' ', '你', '在', '哪裡', '找到', '它', '的']
doc_words2 = ['你', '看', ' ', '這是', '我', '新', '買', '的', '錢包']

invec1 = model.infer_vector(doc_words1)
invec2 = model.infer_vector(doc_words2)
print(invec1)
print(invec2)

# get similarity
# the output docid is supposed to be 0
sims = model.docvecs.most_similar([invec1])
print(sims)

sims = cosine_similarity(invec2.reshape(1, -1), invec2.reshape(1, -1))
print(sims)

[-0.370944   -0.21415496 -0.15595432 -0.33281678  0.23018417 -0.2559593
  0.29821488  0.54231477  0.03856311  0.642508    0.16359109 -0.40134442
 -0.24783419  0.57242703 -0.428072   -0.1313937  -0.826978    0.38550678
  0.44098067 -0.3575736  -0.04831852  0.37350518  0.25847587 -0.09919845
 -0.09718899 -0.18083425  0.30289182  0.31137726 -0.12727624 -0.41865405
  0.01611029  0.00499394  0.25505286 -0.24586713 -0.386765    0.01128664
 -0.30906743  0.14207564  0.05504241 -0.26059055  0.8823103  -0.09638795
 -0.33002147 -0.08997504 -0.32410067  0.43800202  0.00893846  0.42319456
 -0.02953357  0.3060897 ]
[-1.0244675   0.22525074  0.36452842  0.37223774 -0.10813846 -0.08473047
 -0.6506331   0.08327502  0.1664629  -0.17146659 -0.36690986  0.17627233
  0.08673861 -0.15081042 -0.05032846  0.5910078   0.61745477  0.32241046
  0.18303536 -0.56708086 -0.13867296  0.03548172 -0.3764001  -0.20676647
  0.15308632 -0.12855968  0.23481199  0.11043049 -0.11555359 -0.6955886
 -0.4286873   0.14627835 -0